Cual es la app más cara de cada categoría.

# Cargo el csv y hago ajustes necesarios.

In [1]:
! pip install pyspark
! pip install -U -q PyDrive
! apt update
! apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ecdb3e8794b3596dfab772878fcdec81521cf108b38c9b64c17a153612707eb8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.types import IntegerType

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({"id" : "1-oPVltZP-8bpOqQtlRLD6yy6TNasoHLP"})
downloaded.GetContentFile("googleplaystore.csv")

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Skipeo una línea con el nombre de la aplicación respectivo porque está mal parseada.

In [6]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore.csv", header = True, inferSchema = True)
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Comienzo del ejercicio.

Uso take en cada celda para la revisión y sé que no es necesario.

Tomo todas las aplicaciones en cuenta aunque sean gratis porque no me lo específica el ejercicio, y hay categorías sin aplicaciones pagas donde la app más cara es $0, así que muestro una aleatoria.

In [7]:
rdd.filter(lambda x: x.Type == "Paid").map(lambda x: x.Category).distinct().count()

30

In [8]:
rdd.map(lambda x: x.Category).distinct().count()

33

In [9]:
rdd_precios = rdd.map(lambda x: (x.Category, (x.App, float(x.Price.replace("$", "")))))
rdd_precios.take(5)

[('ART_AND_DESIGN', ('Photo Editor & Candy Camera & Grid & ScrapBook', 0.0)),
 ('ART_AND_DESIGN', ('Coloring book moana', 0.0)),
 ('ART_AND_DESIGN',
  ('U Launcher Lite – FREE Live Cool Themes, Hide Apps', 0.0)),
 ('ART_AND_DESIGN', ('Sketch - Draw & Paint', 0.0)),
 ('ART_AND_DESIGN', ('Pixel Draw - Number Art Coloring Book', 0.0))]

In [10]:
rdd_precios.filter(lambda x: x[0] == "LIFESTYLE").reduce(lambda x, y: x if x[1][1] > y[1][1] else y)

('LIFESTYLE', ("I'm Rich - Trump Edition", 400.0))

Las aplicaciones más caras de cada categoria son:

In [11]:
rdd_precios = rdd_precios.reduceByKey(lambda x, y: x if x[1] > y[1] else y)
rdd_precios.collect()

[('ART_AND_DESIGN', ('X Launcher Prime: With OS Style Theme & No Ads', 1.99)),
 ('AUTO_AND_VEHICLES', ('FORD V SERIES CALC - NO LIMIT', 9.99)),
 ('BEAUTY', ('Female Daily', 0.0)),
 ('BOOKS_AND_REFERENCE', ('FN model 1900 pistol explained', 6.49)),
 ('BUSINESS', ('Lean EQ', 89.99)),
 ('COMICS', ('Manga-FR - Anime Vostfr', 0.0)),
 ('COMMUNICATION', ('Z PIVOT', 19.99)),
 ('DATING', ('AMBW Dating App: Asian Men Black Women Interracial', 7.99)),
 ('EDUCATION', ('Fuzzy Numbers: Pre-K Number Foundation', 5.99)),
 ('ENTERTAINMENT', ('My Talking Pet', 4.99)),
 ('EVENTS', ('BP Fitness Lead Scanner', 109.99)),
 ('FINANCE', ('I AM RICH PRO PLUS', 399.99)),
 ('FOOD_AND_DRINK', ('Paprika Recipe Manager', 4.99)),
 ('HEALTH_AND_FITNESS', ('Vital Tones EI Pro', 9.99)),
 ('HOUSE_AND_HOME', ('Santa Fe Sentry', 0.0)),
 ('LIBRARIES_AND_DEMO', ('Socle Commun BO du 23-04-2015', 0.99)),
 ('LIFESTYLE', ("I'm Rich - Trump Edition", 400.0)),
 ('GAME', ('The World Ends With You', 17.99)),
 ('FAMILY', ('I Am Rich 